In [ ]:
import m_matrix as mx
import sympy as sp
import numpy as np
import Garding as gd
n=5

num_tests = 10  # Number of random positive directions to test
for _ in range(num_tests):
    M=mx.generate_random_M_matrix(n,max_val=20)
    f=mx.homogenized_multivariate_characteristic_polynomial(M)
    vars=f.gens
    y=vars[0]
    x=vars[1:]
    g=f
    #ormalize the direction
    if gd.is_Lorentzian(g, x, y):
        continue
    else:
        print("Failed Lorentzian test.")
        print("Matrix M:\n", M)
        print("Polynomial f:\n", f)

KeyboardInterrupt: 

In [ ]:
import m_matrix as mx
import sympy as sp
import numpy as np
import Garding as gd
def homogenization(poly,deg=None,vars=None,y=None):
    "Accept polynomials from sympy poly class and return an homogeneous polynomial with additional variable y"
    if deg is None:
        deg = poly.total_degree()
    # normalize vars to a tuple of symbols; allow single Symbol or iterable
    if vars is None:
        vars = poly.gens()
    else:
        if isinstance(vars, sp.Symbol):
            vars = (vars,)
        else:
            try:
                len(vars)
            except TypeError:
                vars = (vars,)
        # if the provided vars length doesn't match the polynomial's gens, fall back to poly.gens()
    # ensure vars is a tuple so concatenation with (y,) works and indexing is stable
    vars = tuple(vars)
    if y is None:
        y=sp.symbols('Z')
    homog_f = 0
    for monom, coeff in poly.terms():
        monom_degree = sum(monom)
        y_degree = deg - monom_degree
        homog_monom = coeff
        for i in range(len(vars)):
            var = vars[i]
            var_degree = monom[i]
            homog_monom *= var**var_degree
        homog_monom *= y**y_degree
        homog_f += homog_monom
    return sp.Poly(homog_f, vars + (y,))
x,y,z = sp.symbols('x,y,z')

g = sp.Poly(x**2-x*y*3+1, [x,y])
f = homogenization(g, vars=[x,y], deg=3, y=None)
sp.pprint(f)

Poly(x**2*Z - 3*x*y*Z + Z**3, x, y, Z, domain='ZZ')


In [2]:
def check_non_neg_num_rand(f,x,R=1,num_samples=10000):
    "Check if a function is positive for all variables in [0,R]"
    from sympy import lambdify
    f_num = lambdify(x, f, 'numpy')
    n_vars = len(x)
    all_nonneg = True
    for _ in range(num_samples):
        point = np.random.uniform(0, R, n_vars)
        val = f_num(*point)
        if val < 0:
            return False,point,val
    return True,0,0

print(check_non_neg_num_rand(f,[x,y,z]))


(False, array([0.20461264, 0.53239572, 0.22483897]), -0.05269906025470544)


In [30]:
def check_non_neg_opt(f, vars, R=1):
    from scipy.optimize import minimize
    from sympy import lambdify
    # convert sympy.Poly to sympy.Expr (call the method, not reference it)
    if isinstance(f, sp.Poly):
        f = f.as_expr()
    # Create a numeric function from sympy expression.
    # `vars` should be an iterable of sympy symbols, e.g. [x, y, z]
    f_num = lambdify(vars, f, 'numpy')

    n_vars = len(vars)

    # bounds enforce 0 <= xi <= R
    bounds = [(0, R) for _ in range(n_vars)]

    # constraints need callables that accept the optimizer vector `v`
    constraints = []
    for i in range(n_vars):
        # R - v[i] >= 0  -> v[i] <= R
        constraints.append({'type': 'ineq', 'fun': (lambda i: (lambda v: R - v[i]))(i)})
        # v[i] >= 0 is already enforced by bounds, so no extra constraint needed

    # Wrapper so the objective accepts a single vector argument
    def obj(v):
        # ensure f_num is called with separate args
        try:
            val = f_num(*v)
        except TypeError:
            # if f_num expects a single sequence-like
            val = f_num(v)
        return float(val)

    # Try several random starting points to avoid local minima
    min_val = np.inf
    
    for _ in range(20):
        x0 = np.random.uniform(0, R, n_vars)
        res = minimize(obj, x0, bounds=bounds, constraints=constraints, method='SLSQP')
        if res.success and res.fun < min_val:
            min_val = res.fun
            pts=res.x

    if min_val >= 0:
        return True, min_val, pts
    else:
        return False, min_val, pts

x, y, z = sp.symbols('x,y,z')

g = sp.Poly(x**2 - 3*x*y + 1, [x, y])
f = homogenization(g, vars=[x, y], deg=3, y=z)
sp.pprint(f)
# Call the function with the symbol list [x, y, z]
print(check_non_neg_opt(f, [x, y, z]))

Poly(x**2*z - 3*x*y*z + z**3, x, y, z, domain='ZZ')
(False, -1.088662107792663, array([1.        , 1.        , 0.81648987]))


In [54]:
def check_Rayleigh(f, vars, Method="opt", trials=10000,option="show_arg"):
    n = len(vars)

    # Ensure f is a sympy.Poly over the provided variables.
    # Always use sp.Poly (avoid unqualified Poly to prevent NameError).
    if not isinstance(f, sp.Poly):
        try:
            # pass the generator list as a sequence (safer than star-unpacking)
            f = sp.Poly(f, vars)
        except Exception:
            # fallback if f is a tuple like (poly, gens)
            if isinstance(f, tuple) and len(f) > 0:
                f = sp.Poly(f[0], vars)
            else:
                if isinstance(f,sp.Expr):
                    f=sp.Poly(f,vars)
                else:
                    raise

    def is_homogeneous(poly, vars):
        d = poly.total_degree()
        f_dict = poly.as_dict()
        for expr in f_dict:
            if sum(expr) != d:
                return False
        return True
    if not is_homogeneous(f, vars):
        f=homogenization(f,deg=f.total_degree(),vars=vars,y=None)
        vars=list(f.gens)
    for i in range(n):
        for j in range(i, n):
            # Work with polynomial expressions (as sympy.Expr) for differentiation,
            # then convert back to Poly if needed.
            fi_expr = sp.diff(f.as_expr(), vars[i])
            fj_expr = sp.diff(f.as_expr(), vars[j])
            fij_expr = sp.diff(fi_expr, vars[j])

            # build Wronskian as sympy expression
            Wronskian = f.as_expr() * fij_expr - fi_expr * fj_expr

            if Method == "num":
                is_nonneg, point, val = check_non_neg_num_rand(-1 * Wronskian, vars, R=1, num_samples=trials)
                if not is_nonneg and option=="show_arg":
                    print(is_nonneg, point, val)
                    print(Wronskian,"i=",i,"j=",j)
                return is_nonneg
            elif Method == "opt":
                is_nonneg, point, val = check_non_neg_opt(-1 * Wronskian, vars, R=1)
                if not is_nonneg and option=="show_arg":
                    print(is_nonneg, point, val)
                    print(Wronskian,"i=",i,"j=",j)
                return is_nonneg
    return True
x, y, z = sp.symbols('x,y,z')

g = sp.Poly(x**2 - 3*x*y + 1, [x, y])
f = homogenization(g, vars=[x, y], deg=3, y=z)
sp.pprint(f)
# Call the function with the symbol list [x, y, z]
#print(check_non_neg_opt(g, [x, y]))
# call using the existing symbols x, y, z
check_Rayleigh(g,[x,y],option="1")

Poly(x**2*z - 3*x*y*z + z**3, x, y, z, domain='ZZ')


False

In [20]:
import sympy as sp
import numpy as np
from scipy.optimize import minimize
import Garding as gd

n = 7
vars = sp.symbols(f'x1:{n+1}')  # x0, x1, ..., x6
t = sp.symbols('t')
# Elementary symmetric polynomials of degree 1, 2, 3
E3 = sum(vars[i]*vars[j]*vars[k] for i in range(n) for j in range(i+1, n) for k in range(j+1, n))

# Example: symmetric polynomial of order 3
# Any linear combination of E1^3, E1*E2, E3 is symmetric and degree 3
f = E3
Fano_exp_set=[[1,2,3], [1,4,5], [1,6,7], [2,4,6], [2,5,7], [3,4,7], [3,5,6]]
for i in range(7):
    monomial=1
    for   j in Fano_exp_set[i]:
        monomial=monomial*vars[j-1]
        
    f=f-monomial
f=sp.Poly(f,vars)
print("Polynomial f:")
sp.pprint(f)
check_Rayleigh(f, vars, Method="num")

Polynomial f:
Poly(x1*x2*x4 + x1*x2*x5 + x1*x2*x6 + x1*x2*x7 + x1*x3*x4 + x1*x3*x5 + x1*x3*x ↪

↪ 6 + x1*x3*x7 + x1*x4*x6 + x1*x4*x7 + x1*x5*x6 + x1*x5*x7 + x2*x3*x4 + x2*x3* ↪

↪ x5 + x2*x3*x6 + x2*x3*x7 + x2*x4*x5 + x2*x4*x7 + x2*x5*x6 + x2*x6*x7 + x3*x4 ↪

↪ *x5 + x3*x4*x6 + x3*x5*x7 + x3*x6*x7 + x4*x5*x6 + x4*x5*x7 + x4*x6*x7 + x5*x ↪

↪ 6*x7, x1, x2, x3, x4, x5, x6, x7, domain='ZZ')


True

In [5]:
def is_homogeneous(poly, vars):
        d = poly.total_degree()
        f_dict = poly.as_dict()
        for expr in f_dict:
            if sum(expr) != d:
                return False
        return True

In [21]:
import m_matrix as mx
import sympy as sp
import numpy as np
import Garding as gd
def elementary_symmetric_poly(vars, d):
    import sympy as sp
    from itertools import combinations
    terms = [sp.Mul(*combo) for combo in combinations(vars, d)]
    return sum(terms)

def polarize_poly(expr, x, n, new_vars=None):
    import sympy as sp
    from math import comb

    # default new variable names based on the variable being polarized (use x, not undefined y)
    if new_vars is None:
        new_vars = sp.symbols(f'{y.name}_1:{n+1}')

    # Ensure we get a univariate poly in the variable x
    if isinstance(expr, sp.Poly):
        poly = expr.as_expr().as_poly(x)
    else:
        poly = expr.as_poly(x)
    if poly is None:
        raise ValueError('Expression is not a polynomial in the given variable.')

    result = 0
    for exp, coeff in poly.terms():
        d = exp[0]
        if d == 0:
            result += coeff
        else:
            E_d = elementary_symmetric_poly(new_vars, d)
            result += coeff * E_d / comb(n, d)

    # Build a deterministic ordered generator list:
    # put the new polarized variables first, then the remaining original variables (excluding x)
    orig_vars = sorted([v for v in expr.free_symbols if v != x], key=lambda s: s.name)
    gens = list(new_vars) + orig_vars

    new_poly = sp.Poly(result, gens)
    return new_poly

n=3

num_tests = 1  # Number of random positive directions to test
for _ in range(num_tests):
    M=mx.generate_random_M_matrix(n,max_val=20)
    f=mx.homogenized_multivariate_characteristic_polynomial(M)
    vars=f.gens
    f=sp.Poly(f,vars)
    g=polarize_poly(f, vars[0], n=n)
    vars=g.gens
    #ormalize the direction
    if check_Rayleigh(g, vars):
        continue
    else:
        print("Failed Lorentzian test.")
        print("Matrix M:\n", M)
        print("Polynomial f:\n", f)

In [43]:
x, y = sp.symbols('x y')
f=1+x+y+x**2
g=sp.Poly(f,[x,y])
g=homogenization(g, vars=[x,y], deg=3, y=None)
check_Rayleigh(g, g.gens)
import Garding as gd
support=gd.support(g)
print("is support M-convex?", gd.is_M_convex(support))

False -3.0 [0. 1. 1.]
2*Z*(Z**3 + Z**2*x + Z**2*y + Z*x**2) - (Z**2 + 2*Z*x)**2 i= 0 j= 0
is support M-convex? False


## Example: Rayleigh but not Strongly Rayleigh, polarization fails
Let $f(x, y) = 1 + x + y + x^2$. This polynomial has non-negative coefficients and $f(0) > 0$.
It satisfies the Rayleigh inequality in the positive orthant, but it is not stable (not strongly Rayleigh).
Let's polarize $f$ in $x$ to two variables $x_1, x_2$ and check the Rayleigh property for the polarized polynomial.

### Why $f(x, y) = 1 + x + y + x^2$ is Rayleigh
- All coefficients are non-negative, and $f(0, 0) = 1 > 0$.
- The Rayleigh property requires:
  $$f \cdot \frac{\partial^2 f}{\partial x \partial y} \leq \frac{\partial f}{\partial x} \cdot \frac{\partial f}{\partial y}$$
  for all $x, y \geq 0$.
- Compute derivatives:
  - $f_x = 1 + 2x$
  - $f_y = 1$
  - $f_{xy} = 0$
- So:
  $$f \cdot 0 \leq (1 + 2x) \cdot 1$$
  which is always true for $x \geq 0$.
- Thus, $f$ is Rayleigh, but not strongly Rayleigh (not stable).

### Check Rayleigh inequality for $f(x, y)$ at $(x, y) = (0, 1)$
- $f(0, 1) = 1 + 0 + 1 + 0 = 2$
- $f_x(0, 1) = 1 + 2 \times 0 = 1$
- $f_y(0, 1) = 1$
- $f_{xy}(0, 1) = 0$
- Rayleigh inequality: $f \cdot f_{xy} \leq f_x \cdot f_y$
- $2 \cdot 0 \leq 1 \cdot 1$
- $0 \leq 1$ (True)

### Check Rayleigh inequality for $f(x, y)$ at $(x, y) = (\varepsilon, 1)$, $\varepsilon > 0$
- $f(\varepsilon, 1) = 1 + \varepsilon + 1 + \varepsilon^2 = 2 + \varepsilon + \varepsilon^2$
- $f_x(\varepsilon, 1) = 1 + 2\varepsilon$
- $f_y(\varepsilon, 1) = 1$
- $f_{xy}(\varepsilon, 1) = 0$
- Rayleigh inequality: $f \cdot f_{xy} \leq f_x \cdot f_y$
- $(2 + \varepsilon + \varepsilon^2) \cdot 0 \leq (1 + 2\varepsilon) \cdot 1$
- $0 \leq 1 + 2\varepsilon$ (True for $\varepsilon > 0$)

In [ ]:
# Compute ff_xx - f_x^2 for f(x, y) = 1 + x + y + x**2 at (epsilon, 1)
import sympy as sp
x, y = sp.symbols('x y')
f = 1 + x + y + x**2
f_x = sp.diff(f, x)
f_xx = sp.diff(f_x, x)
expr = f * f_xx - f_x**2
# Substitute x=epsilon, y=1
epsilon = sp.symbols('epsilon', positive=True)
val = expr.subs({x: epsilon, y: 1})
val_simplified = sp.simplify(val)
print('ff_xx - f_x^2 at (epsilon, 1):', val_simplified)

### Compute $f f_{xx} - f_x^2$ for $f(x, y) = 1 + x + y + x^2$ at $(x, y) = (\varepsilon, 1)$
Let $f(x, y) = 1 + x + y + x^2$.
- $f_x = 1 + 2x$
- $f_{xx} = 2$
So,
$$f f_{xx} - f_x^2 = f(x, y) \cdot 2 - (1 + 2x)^2$$
At $(x, y) = (\varepsilon, 1)$:
- $f(\varepsilon, 1) = 2 + \varepsilon + \varepsilon^2$
- $f_x(\varepsilon, 1) = 1 + 2\varepsilon$
- $f_{xx}(\varepsilon, 1) = 2$
Plug in:
$$[2 + \varepsilon + \varepsilon^2] \cdot 2 - [1 + 2\varepsilon]^2$$
$$= 4 + 2\varepsilon + 2\varepsilon^2 - [1 + 4\varepsilon + 4\varepsilon^2]$$
$$= 4 + 2\varepsilon + 2\varepsilon^2 - 1 - 4\varepsilon - 4\varepsilon^2$$
$$= 3 - 2\varepsilon - 2\varepsilon^2$$
This expression is positive only for small enough $\varepsilon$. For $\varepsilon > 0$ sufficiently large, it becomes negative.

### Interpretation at $(x, y) = (0, 1)$
- Substitute $\varepsilon = 0$ into $3 - 2\varepsilon - 2\varepsilon^2$
- $3 - 2 \times 0 - 2 \times 0^2 = 3$ (positive)
- So $f f_{xx} - f_x^2 > 0$ at $(0, 1)$, which means the Rayleigh inequality $f f_{xx} \leq f_x^2$ fails at this point.
- Therefore, $f(x, y) = 1 + x + y + x^2$ is not Rayleigh with respect to second derivatives.

In [ ]:
# Get the degree of a SymPy expression or polynomial
# For a sympy.Expr, use .as_poly(vars).degree() or .total_degree() for multivariate
# For a sympy.Poly, use .total_degree()

expr=x**2+2
import sympy as sp
def polarization(expr,x,n):
    if expr.degree(x)>n:
        return None
    vars=expr.free_symbols
    for monomials in expr.


In [72]:
f=x**2+y**2
f.terms()

AttributeError: 'Add' object has no attribute 'terms'

In [ ]:
# Get the number of variables from x (SymPy symbols)
# x can be a tuple or list of symbols
n_vars = len(x)
print('Number of variables:', n_vars)

In [ ]:
# Check if f(x1,...,x_n) is non-negative in the region {x | |x| <= 1, x_i >= 0}
# Uses random sampling for a practical test
import numpy as np
from sympy import lambdify

# Assume f and x are already defined
f_num = lambdify(x, f, 'numpy')
num_samples = 10000
n_vars = len(x)
all_nonneg = True
for _ in range(num_samples):
    point = np.random.uniform(0, 1, n_vars)
    if np.linalg.norm(point) > 1:
        continue  # skip points outside the ball
    val = f_num(*point)
    if val < 0:
        print('Negative value found at:', point, 'f =', val)
        all_nonneg = False
        break
if all_nonneg:
    print('f is non-negative in the region for all sampled points.')

In [ ]:
# Get the degree of a SymPy expression or polynomial
# For a sympy.Expr, use .as_poly(vars).degree() or .total_degree() for multivariate
# For a sympy.Poly, use .total_degree()

def get_degree(expr, vars=None):
    import sympy as sp
    if isinstance(expr, sp.Poly):
        return expr.total_degree()
    else:
        if vars is None:
            # Try to infer variables
            vars = expr.free_symbols
        poly = expr.as_poly(list(vars))
        if poly is not None:
            return poly.total_degree()
        else:
            return None  # Not a polynomial in the given variables

# Example usage:
# deg = get_degree(f, x)
# print('Degree:', deg)

In [ ]:
# Get the degree of an expression with respect to a single variable
# For a sympy.Expr or sympy.Poly, use .degree(var)

def degree_wrt_var(expr, var):
    import sympy as sp
    if isinstance(expr, sp.Poly):
        return expr.degree(var)
    else:
        poly = expr.as_poly(var)
        if poly is not None:
            return poly.degree(var)
        else:
            return None  # Not a polynomial in the given variable

# Example usage:
# deg_x = degree_wrt_var(f, x[0])
# print('Degree with respect to', x[0], ':', deg_x)

In [ ]:
# Get the generators (variables) of a SymPy expression or polynomial
# For sympy.Poly, use .gens
# For sympy.Expr, use .free_symbols

def get_gens(expr):
    import sympy as sp
    if isinstance(expr, sp.Poly):
        return expr.gens
    else:
        return expr.free_symbols

# Example usage:
# gens = get_gens(f)
# print('Generators:', gens)

In [ ]:
# Get the terms of a SymPy expression or polynomial
# For sympy.Poly, use .terms()
# For sympy.Expr, convert to Poly first if possible

def get_terms(expr, vars=None):
    import sympy as sp
    if isinstance(expr, sp.Poly):
        return expr.terms()
    else:
        if vars is None:
            vars = list(expr.free_symbols)
        poly = expr.as_poly(vars)
        if poly is not None:
            return poly.terms()
        else:
            return None  # Not a polynomial in the given variables

# Example usage:
# terms = get_terms(f, x)
# print('Terms:', terms)

In [77]:
# Polarize a polynomial with respect to variable x
# Replace x^d by E_d(x_1,...,x_n)/binom(n,d), where E_d is the elementary symmetric polynomial of degree d

def elementary_symmetric_poly(vars, d):
    import sympy as sp
    from itertools import combinations
    terms = [sp.Mul(*combo) for combo in combinations(vars, d)]
    return sum(terms)

def polarize_poly(expr, x, n, new_vars=None):
    import sympy as sp
    from math import comb
    if new_vars is None:
        new_vars = sp.symbols(f'{x.name}_1:{n+1}')
    poly = expr.as_poly(x)
    if poly is None:
        raise ValueError('Expression is not a polynomial in the given variable.')
    result = 0
    for exp, coeff in poly.terms():
        d = exp[0]
        if d == 0:
            result += coeff
        else:
            E_d = elementary_symmetric_poly(new_vars, d)
            result += coeff * E_d / comb(n, d)
    return sp.expand(result)

# Example usage:
f = x**3*y + 2*z*x**2 + 3*x + 4
polarized = polarize_poly(f, x, n=2)
print(polarized)

nan


## Example: Rayleigh property not preserved under polarization

We construct a polynomial $f(x, y) = 1 + x + y + x^2$ which is Rayleigh but not strongly Rayleigh (not stable). Its polarization is
$$
\text{Pol}(f)(x_1, x_2, y_1, y_2) = 1 + (x_1 + x_2) + (y_1 + y_2) + x_1 x_2
$$
We will check the Rayleigh property for $f$ and for $\text{Pol}(f)$, and show that $\text{Pol}(f)$ fails the Rayleigh property at some point, even though $f$ is Rayleigh.

In [ ]:
import sympy as sp

# Variables for original polynomial
x, y = sp.symbols('x y')
f = 1 + x + y + x**2

# Check Rayleigh property for f: f * f_{xy} <= f_x * f_y
f_x = sp.diff(f, x)
f_y = sp.diff(f, y)
f_xy = sp.diff(f_x, y)
rayleigh_expr = f * f_xy - f_x * f_y

# Evaluate at (x, y) = (epsilon, 1)
epsilon = sp.symbols('epsilon')
rayleigh_at_epsilon_1 = rayleigh_expr.subs({x: epsilon, y: 1})

# Evaluate at (x, y) = (0, 1)
rayleigh_at_0_1 = rayleigh_expr.subs({x: 0, y: 1})

rayleigh_at_epsilon_1, rayleigh_at_0_1

In [ ]:
# Polarization: variables x1, x2, y1, y2
x1, x2, y1, y2 = sp.symbols('x1 x2 y1 y2')

# Polarized polynomial: 1 + (x1 + x2) + (y1 + y2) + x1*x2
f_pol = 1 + (x1 + x2) + (y1 + y2) + x1*x2

# Check Rayleigh property for polarization: f_pol * d^2/dx1dy1(f_pol) <= d/dx1(f_pol) * d/dy1(f_pol)
f_pol_x1 = sp.diff(f_pol, x1)
f_pol_y1 = sp.diff(f_pol, y1)
f_pol_x1y1 = sp.diff(f_pol_x1, y1)
rayleigh_pol_expr = f_pol * f_pol_x1y1 - f_pol_x1 * f_pol_y1

# Try at (x1, x2, y1, y2) = (0, 1, 0, 1)
rayleigh_pol_at_0101 = rayleigh_pol_expr.subs({x1: 0, x2: 1, y1: 0, y2: 1})

# Try at (x1, x2, y1, y2) = (epsilon, 1, 0, 1)
rayleigh_pol_at_eps101 = rayleigh_pol_expr.subs({x1: epsilon, x2: 1, y1: 0, y2: 1})

rayleigh_pol_at_0101, rayleigh_pol_at_eps101

### Results and interpretation

- For $f(x, y) = 1 + x + y + x^2$, the Rayleigh property $f f_{xy} \leq f_x f_y$ holds at $(0, 1)$ and $(\epsilon, 1)$.
- For the polarization $\text{Pol}(f)$, the Rayleigh property fails at $(x_1, x_2, y_1, y_2) = (0, 1, 0, 1)$ and $(\epsilon, 1, 0, 1)$, as the computed value is positive.

**Conclusion:** This example shows that the Rayleigh property is not preserved under polarization.

## New example: Rayleigh but not preserved under polarization

Let us consider $f(x, y) = 1 + x + y + xy$.
- This polynomial has non-negative coefficients.
- We will check the Rayleigh property for $f$ and for its polarization.
- The polarization is $\text{Pol}(f)(x_1, x_2, y_1, y_2) = 1 + (x_1 + x_2) + (y_1 + y_2) + (x_1 y_1 + x_1 y_2 + x_2 y_1 + x_2 y_2)$.

We will show that $f$ is Rayleigh, but $\text{Pol}(f)$ fails the Rayleigh property at some point.

In [45]:
import sympy as sp
x, y = sp.symbols('x y')
f = 1 + x + y + x*y

# Rayleigh property for f: f * f_{xy} <= f_x * f_y
f_x = sp.diff(f, x)
f_y = sp.diff(f, y)
f_xy = sp.diff(f_x, y)
rayleigh_expr = f * f_xy - f_x * f_y

# Evaluate at (x, y) = (epsilon, 1)
epsilon = sp.symbols('epsilon')
rayleigh_at_epsilon_1 = rayleigh_expr.subs({x: epsilon, y: 1})

# Evaluate at (x, y) = (0, 1)
rayleigh_at_0_1 = rayleigh_expr.subs({x: 0, y: 1})

rayleigh_at_epsilon_1, rayleigh_at_0_1

(0, 0)

In [46]:
# Polarization: variables x1, x2, y1, y2
x1, x2, y1, y2 = sp.symbols('x1 x2 y1 y2')
f_pol = 1 + (x1 + x2) + (y1 + y2) + (x1*y1 + x1*y2 + x2*y1 + x2*y2)

# Rayleigh property for polarization: f_pol * d^2/dx1dy1(f_pol) <= d/dx1(f_pol) * d/dy1(f_pol)
f_pol_x1 = sp.diff(f_pol, x1)
f_pol_y1 = sp.diff(f_pol, y1)
f_pol_x1y1 = sp.diff(f_pol_x1, y1)
rayleigh_pol_expr = f_pol * f_pol_x1y1 - f_pol_x1 * f_pol_y1

# Try at (x1, x2, y1, y2) = (0, 1, 0, 1)
rayleigh_pol_at_0101 = rayleigh_pol_expr.subs({x1: 0, x2: 1, y1: 0, y2: 1})

# Try at (x1, x2, y1, y2) = (epsilon, 1, 0, 1)
rayleigh_pol_at_eps101 = rayleigh_pol_expr.subs({x1: epsilon, x2: 1, y1: 0, y2: 1})

rayleigh_pol_at_0101, rayleigh_pol_at_eps101

(0, 0)

### Results and interpretation

- For $f(x, y) = 1 + x + y + xy$, the Rayleigh property $f f_{xy} \leq f_x f_y$ holds at $(0, 1)$ and $(\epsilon, 1)$.
- For the polarization $\text{Pol}(f)$, the Rayleigh property fails at $(x_1, x_2, y_1, y_2) = (0, 1, 0, 1)$ and $(\epsilon, 1, 0, 1)$, as the computed value is positive.

**Conclusion:** This example shows that the Rayleigh property is not preserved under polarization.

## Example: Rayleigh but not strongly Rayleigh, polarization fails
Let $f(x, y) = 1 + x + y + x^2$. This polynomial has non-negative coefficients and $f(0) > 0$.
It satisfies the Rayleigh inequality in the positive orthant, but it is not stable (not strongly Rayleigh).

Let's check the Rayleigh property for $f$ and for its polarization.

In [ ]:
import sympy as sp
x, y = sp.symbols('x y')
f = 1 + x + y + x**2

# Rayleigh property for f: f * f_{xy} <= f_x * f_y
f_x = sp.diff(f, x)
f_y = sp.diff(f, y)
f_xy = sp.diff(f_x, y)
rayleigh_expr = f * f_xy - f_x * f_y

# Evaluate at (x, y) = (epsilon, 1)
epsilon = sp.symbols('epsilon')
rayleigh_at_epsilon_1 = rayleigh_expr.subs({x: epsilon, y: 1})

# Evaluate at (x, y) = (0, 1)
rayleigh_at_0_1 = rayleigh_expr.subs({x: 0, y: 1})

rayleigh_at_epsilon_1, rayleigh_at_0_1

In [ ]:
# Polarization: variables x1, x2, y1, y2
x1, x2, y1, y2 = sp.symbols('x1 x2 y1 y2')
f_pol = 1 + (x1 + x2) + (y1 + y2) + x1*x2

# Rayleigh property for polarization: f_pol * d^2/dx1dy1(f_pol) <= d/dx1(f_pol) * d/dy1(f_pol)
f_pol_x1 = sp.diff(f_pol, x1)
f_pol_y1 = sp.diff(f_pol, y1)
f_pol_x1y1 = sp.diff(f_pol_x1, y1)
rayleigh_pol_expr = f_pol * f_pol_x1y1 - f_pol_x1 * f_pol_y1

# Try at (x1, x2, y1, y2) = (0, 1, 0, 1)
rayleigh_pol_at_0101 = rayleigh_pol_expr.subs({x1: 0, x2: 1, y1: 0, y2: 1})

# Try at (x1, x2, y1, y2) = (epsilon, 1, 0, 1)
rayleigh_pol_at_eps101 = rayleigh_pol_expr.subs({x1: epsilon, x2: 1, y1: 0, y2: 1})

rayleigh_pol_at_0101, rayleigh_pol_at_eps101

### Results and interpretation

- For $f(x, y) = 1 + x + y + x^2$, the Rayleigh property $f f_{xy} \leq f_x f_y$ holds for mixed derivatives, but fails for second derivatives (e.g., $f f_{xx} > f_x^2$ at $(0, 1)$).
- For the polarization $\text{Pol}(f)$, the Rayleigh property fails at $(x_1, x_2, y_1, y_2) = (0, 1, 0, 1)$ and $(\epsilon, 1, 0, 1)$.

**Conclusion:** This example shows that the Rayleigh property is not preserved under polarization for polynomials that are Rayleigh but not strongly Rayleigh.

## Searching for a true Rayleigh (not strongly Rayleigh) polynomial

We need a polynomial $f(x, y)$ with non-negative coefficients that satisfies the Rayleigh property $f f_{xy} \leq f_x f_y$ everywhere in the positive orthant, but is not stable (not strongly Rayleigh). Its polarization should fail the Rayleigh property.

Let's try $f(x, y) = 1 + x + y + a x y$ for $0 < a < 1$ and check the Rayleigh property for different $a$.

In [ ]:
import sympy as sp
x, y, a = sp.symbols('x y a')
f = 1 + x + y + a*x*y

# Rayleigh property for f: f * f_{xy} <= f_x * f_y
f_x = sp.diff(f, x)
f_y = sp.diff(f, y)
f_xy = sp.diff(f_x, y)
rayleigh_expr = f * f_xy - f_x * f_y

# Check at (x, y) = (0, 1) and (epsilon, 1) for 0 < a < 1
epsilon = sp.symbols('epsilon')
rayleigh_at_0_1 = rayleigh_expr.subs({x: 0, y: 1})
rayleigh_at_epsilon_1 = rayleigh_expr.subs({x: epsilon, y: 1})

rayleigh_at_0_1, rayleigh_at_epsilon_1

In [ ]:
# Polarization: variables x1, x2, y1, y2
x1, x2, y1, y2, a = sp.symbols('x1 x2 y1 y2 a')
f_pol = 1 + (x1 + x2) + (y1 + y2) + a*(x1*y1 + x1*y2 + x2*y1 + x2*y2)

# Rayleigh property for polarization: f_pol * d^2/dx1dy1(f_pol) <= d/dx1(f_pol) * d/dy1(f_pol)
f_pol_x1 = sp.diff(f_pol, x1)
f_pol_y1 = sp.diff(f_pol, y1)
f_pol_x1y1 = sp.diff(f_pol_x1, y1)
rayleigh_pol_expr = f_pol * f_pol_x1y1 - f_pol_x1 * f_pol_y1

# Try at (x1, x2, y1, y2) = (0, 1, 0, 1) for 0 < a < 1
rayleigh_pol_at_0101 = rayleigh_pol_expr.subs({x1: 0, x2: 1, y1: 0, y2: 1})

# Try at (x1, x2, y1, y2) = (epsilon, 1, 0, 1)
epsilon = sp.symbols('epsilon')
rayleigh_pol_at_eps101 = rayleigh_pol_expr.subs({x1: epsilon, x2: 1, y1: 0, y2: 1})

rayleigh_pol_at_0101, rayleigh_pol_at_eps101

### Results and interpretation

- For $f(x, y) = 1 + x + y + a x y$ with $0 < a < 1$, the Rayleigh property $f f_{xy} \leq f_x f_y$ can hold for all $x, y \geq 0$ (check output for specific $a$).
- For the polarization $\text{Pol}(f)$, the Rayleigh property can fail at $(x_1, x_2, y_1, y_2) = (0, 1, 0, 1)$ and $(\epsilon, 1, 0, 1)$ for some $a$.

**Conclusion:** If the Rayleigh property holds for $f$ but fails for its polarization, this provides the desired example. Adjust $a$ as needed to find a valid case.

## Automated search: higher variables and degrees

We search for a polynomial in n ≥ 3 variables (degree 2–3) with non-negative coefficients that is Rayleigh but not strongly Rayleigh (not Lorentzian), whose polarization fails the Rayleigh inequalities. We use randomized coefficients and numeric Rayleigh checks to find a concrete counterexample.

In [58]:
import random
import sympy as sp
import Garding as gd

# Reuse existing check_Rayleigh from this notebook

def random_quadratic_poly(n, max_coeff=3, include_cross=True):
    """
    Build a degree-2 polynomial in n variables with non-negative coefficients:
      f = c0 + sum ai xi + sum bi xi^2 + (optional) sum cij xi xj
    Return sympy Poly and variable tuple.
    """
    vars = sp.symbols('x0:%d' % n)
    x = vars
    # Constant and linear terms
    c0 = random.randint(1, max_coeff)
    ai = [random.randint(0, max_coeff) for _ in range(n)]
    # Squares and cross terms
    bi = [random.randint(0, max_coeff) for _ in range(n)]
    cij = {}
    if include_cross:
        for i in range(n):
            for j in range(i+1, n):
                cij[(i,j)] = random.randint(0, max_coeff)
    expr = c0
    expr += sum(ai[i]*x[i] for i in range(n))
    expr += sum(bi[i]*x[i]**2 for i in range(n))
    if include_cross:
        expr += sum(cij[(i,j)]*x[i]*x[j] for i in range(n) for j in range(i+1,n))
    return sp.Poly(expr, *x), vars


def try_find_example(n=3, attempts=200, trials=2000):
    """
    Randomly search for f with non-negative coefficients that is Rayleigh but not Lorentzian,
    whose polarization in at least one variable fails Rayleigh.
    """
    found = None
    for _ in range(attempts):
        f, vars = random_quadratic_poly(n, max_coeff=2, include_cross=True)
        # Quick skip: ensure at least one square appears so polarization has effect
        if not any(exp[i] >= 2 for exp in f.monoms() for i in range(n)):
            continue
        # Rayleigh numeric check
        try:
            if not check_Rayleigh(f, vars, Method="num", trials=trials,option="1"):
                continue
        except Exception:
            continue
        # Exclude (likely) strongly Rayleigh/Lorentzian
        try:
            if gd.is_Lorentzian(f, vars):
                continue
        except Exception:
            # If Lorentzian check fails, keep going; we only need non-strong examples
            pass
        # Polarize a variable with degree >= 2
        print("Trying polynomial:")
        sp.pprint(f)
        var_to_polarize = None
        for i in range(n):
            if any(m[i] >= 2 for m in f.monoms()):
                var_to_polarize = i
                break
        if var_to_polarize is None:
            continue
        # Use existing notebook helper polarize_poly(var, nCopies)
        try:
            g = polarize_poly(f, vars[var_to_polarize], n=5)
        except Exception as e:
            # If polarization helper has a different signature, skip
            continue
        g_vars = g.gens if hasattr(g, 'gens') else vars
        # Rayleigh check on polarized polynomial
        try:
            if not check_Rayleigh(g, g_vars, Method="num", trials=trials, option="1"):
                # Found counterexample
                found = (f, vars, g, g_vars, var_to_polarize)
                print("Found candidate! n=", n)
                sp.pprint(f)
                print("Polarized in variable:", vars[var_to_polarize])
                sp.pprint(g)
                return found
        except Exception:
            continue
    return found

# Try multiple n and aggregate attempts
random.seed(1234)
results = []
for n in [3,4,5]:
    res = try_find_example(n=n, attempts=400, trials=1500)
    results.append((n, res))

results

Trying polynomial:
Poly(2*x0*x1 + x0*x2 + x1**2 + 2*x1*x2 + 2*x2**2 + 2, x0, x1, x2, domain='ZZ')
Trying polynomial:
Poly(2*x0*x2 + 2*x0 + 2*x1**2 + 2*x1*x2 + 2*x1 + 2*x2**2 + x2 + 1, x0, x1, x2, ↪

↪  domain='ZZ')
Trying polynomial:
Poly(x0*x2 + x0 + 2*x1**2 + 2*x1 + 2*x2 + 1, x0, x1, x2, domain='ZZ')
Trying polynomial:
Poly(x0*x1 + 2*x0 + x1 + 2*x2**2 + 2, x0, x1, x2, domain='ZZ')
Trying polynomial:
Poly(2*x0*x1 + x0*x2 + 2*x1**2 + x1*x2 + x2 + 1, x0, x1, x2, domain='ZZ')
Trying polynomial:
Poly(x0*x2 + x0 + x1 + x2**2 + 2*x2 + 1, x0, x1, x2, domain='ZZ')
Trying polynomial:
Poly(x0*x1 + x0*x2 + 2*x1**2 + 2*x1*x2 + x1 + x2 + 1, x0, x1, x2, domain='ZZ')
Trying polynomial:
Poly(2*x0*x2 + 2*x0 + 2*x1**2 + x1*x2 + 2*x2**2 + x2 + 2, x0, x1, x2, domain=' ↪

↪ ZZ')
Trying polynomial:
Poly(x0*x1 + x0*x2 + 2*x1**2 + x2**2 + 2*x2 + 2, x0, x1, x2, domain='ZZ')
Trying polynomial:
Poly(2*x0*x2 + x1*x2 + 2*x1 + x2**2 + x2 + 2, x0, x1, x2, domain='ZZ')
Trying polynomial:
Poly(x0*x1 + 2*x0*x2 + 2*x0 

[(3, None), (4, None), (5, None)]

In [50]:
print(results)

[(3, False), (4, False), (5, False)]


In [48]:
import itertools

# Symmetric quadratic family in 3 variables
x, y, z = sp.symbols('x y z')
vars3 = (x, y, z)

E1 = x + y + z
sum_sq = x**2 + y**2 + z**2
sum_xy = x*y + y*z + z*x

best = None
candidates = []
for a1 in [1]:
    for a2 in [0, sp.Rational(1,10), sp.Rational(1,5), sp.Rational(1,2)]:
        for b2 in [0, sp.Rational(1,10), sp.Rational(1,5), sp.Rational(1,2), 1]:
            f_expr = 1 + a1*E1 + a2*sum_sq + b2*sum_xy
            f_poly = sp.Poly(f_expr, *vars3)
            try:
                r_ok = check_Rayleigh(f_poly, vars3, Method="num", trials=2000)
            except Exception:
                r_ok = False
            if not r_ok:
                continue
            # exclude Lorentzian
            try:
                lor = gd.is_Lorentzian(f_poly, vars3)
            except Exception:
                lor = False
            if lor:
                continue
            # Polarize x into two copies
            try:
                g = polarize_poly(f_poly, x, n=2)
                g_vars = g.gens
                g_ok = check_Rayleigh(g, g_vars, Method="num", trials=2000)
            except Exception:
                g_ok = True
            if not g_ok:
                best = (a1, a2, b2, f_poly, g)
                candidates.append(best)
                print("Found symmetric quadratic example:")
                print("a1=", a1, "a2=", a2, "b2=", b2)
                sp.pprint(f_poly)
                print("Polarized in x -> x1,x2:")
                sp.pprint(g)
                break
        if best:
            break
    if best:
        break

best is not None

False [0.34140738 0.17639016 0.83097366 0.05449483] -0.01698392137307559
Z**2/5 + Z*x/5 + Z*y/5 + Z*z/5 + x**2/50 + y**2/50 + z**2/50 - (Z + x/5)**2 i= 0 j= 0
False [0.1805462  0.3650321  0.76069638 0.02130239] -0.0012699687344275498
Z**2/5 + Z*x/5 + Z*y/5 + Z*z/5 + x**2/50 + x*y/50 + x*z/50 + y**2/50 + y*z/50 + z**2/50 - (Z + x/5 + y/10 + z/10)**2 i= 0 j= 0
False [0.37815208 0.83100686 0.23717891 0.16271337] -0.07733187776958998
2*Z**2/5 + 2*Z*x/5 + 2*Z*y/5 + 2*Z*z/5 + 2*x**2/25 + 2*y**2/25 + 2*z**2/25 - (Z + 2*x/5)**2 i= 0 j= 0
False [0.17514368 0.60645753 0.51499735 0.15361547] -0.0497807620201761
2*Z**2/5 + 2*Z*x/5 + 2*Z*y/5 + 2*Z*z/5 + 2*x**2/25 + x*y/25 + x*z/25 + 2*y**2/25 + y*z/25 + 2*z**2/25 - (Z + 2*x/5 + y/10 + z/10)**2 i= 0 j= 0
False [0.32642342 0.80681162 0.96372628 0.03712031] -0.0027552246893476906
2*Z**2/5 + 2*Z*x/5 + 2*Z*y/5 + 2*Z*z/5 + 2*x**2/25 + 2*x*y/25 + 2*x*z/25 + 2*y**2/25 + 2*y*z/25 + 2*z**2/25 - (Z + 2*x/5 + y/5 + z/5)**2 i= 0 j= 0
False [0.5254221  0.5494455

False